# Amount to Euro conversion

Purpose of this notebook: To convert the "amount" column from local currencis to Euro. Since the amount column

Data paths:
- Input: Basic Df
- Output: Overwrites Basic DF

In [14]:
import pandas as pd
import requests

# URL of the API
url = 'https://api.frankfurter.app/2024-09-26'

# Fetch the data from the API using requests
response = requests.get(url)
data = response.json()
exchange_rates = pd.DataFrame(list(data['rates'].items()), columns=['Currency', 'Rate']) # convert data innto pandas dataframe
exchange_rates.head()

,Currency,Rate
0,AUD,1.6217
1,BGN,1.9558
2,BRL,6.0372
3,CAD,1.5025
4,CHF,0.9452


## Note:

The Frankfurter API provides rate EUR - Currency:
Example: CHF; Rate: 0.9452 --> 1 Euro = 0.9452 CHF

Source for other conversions: https://www.xe.com/currencyconverter/


In [15]:
#load data
df = pd.read_csv('data/invoices_basic.csv')

unique_currency = df['currency'].unique()

# Check if all unique_currencies are in exchange rates and print the ones that are not
for currency in unique_currency:
    if currency not in exchange_rates['Currency'].unique():
        print(currency)

EUR
RSD


In the cell above we found that there is not exchange rate for the Vietnamese Dong and the Serbian Dinar provided by the "Frankfurther API". The remaining exchange rates were taken from yahoo finance

In [16]:
#RSD = Serbian Dinar: take exchange rate from EUR to RSD on 2024-09-26
EUR_to_RSD = 117.0955

# VND = Vietnamese Dong: take exchange rate from EUR to VND on 2024-09-26
EUR_to_VND = 27413.172

# Create a DataFrame with the new entries
new_entries = pd.DataFrame([
    {'Currency': 'RSD', 'Rate': EUR_to_RSD},
    {'Currency': 'VND', 'Rate': EUR_to_VND},
    {'Currency': 'EUR', 'Rate': 1}
])

# Concatenate the new entries with the existing DataFrame
exchange_rates = pd.concat([exchange_rates, new_entries], ignore_index=True)

# Check if all unique_currencies are in exchange rates
for currency in unique_currency:
    if currency not in exchange_rates['Currency'].unique():
        print(currency)

## Creating new column: "amount" Euro

In [17]:
# Merge df with exchange_rates on the currency column
df = df.merge(exchange_rates, left_on='currency', right_on='Currency', how='left')

# Calculate the amount in euros
df['amount_euro'] = df['amount'] / df['Rate']

df

,amount,currency,payment_term,customer_id,receipt_date,due_date,weighted_payment_date,Currency,Rate,amount_euro
0,1000,USD,60,abc123,2023-01-01,2023-03-02,2023-03-20,USD,1.11550,896.458987
1,1500,EUR,15,acb345,2023-01-01,2023-01-16,2023-01-25,EUR,1.00000,1500.000000
2,2000,GBP,30,xyz678,2023-01-01,2023-01-31,2023-01-25,GBP,0.83428,2397.276694
3,2500,RSD,30,abc987,2023-01-01,2023-01-01,2023-01-01,RSD,117.09550,21.350095


## Compare distributions



In [18]:
print(df['amount'].describe())
print("----------------")
print(df['amount_euro'].describe())

count       4.000000
mean     1750.000000
std       645.497224
min      1000.000000
25%      1375.000000
50%      1750.000000
75%      2125.000000
max      2500.000000
Name: amount, dtype: float64
----------------
count       4.000000
mean     1203.771444
std      1000.794240
min        21.350095
25%       677.681764
50%      1198.229494
75%      1724.319173
max      2397.276694
Name: amount_euro, dtype: float64


In [19]:
# save df
df.to_csv("data/invoices_basic.csv", index=False)